In [1]:
import pandas as pd

# Load the final dataset
final_df = pd.read_csv("/content/final_output.csv")  # Adjust path if needed


In [2]:
# Define required daily intake
required_nutrients = {
    'calories': 2000,
    'protein': 50,
    'fat': 70,
    'carbohydrates': 310
}

# Create working copy
gap_df = final_df.copy()

# Compute nutrient gaps
gap_df['calories_gap'] = required_nutrients['calories'] - gap_df['calories']
gap_df['protein_gap'] = required_nutrients['protein'] - gap_df['protein']
gap_df['fat_gap'] = required_nutrients['fat'] - gap_df['fat']
gap_df['carbohydrates_gap'] = required_nutrients['carbohydrates'] - gap_df['carbohydrates']

# Flag deficiencies
gap_df['calories_deficiency'] = gap_df['calories'] < required_nutrients['calories']
gap_df['protein_deficiency'] = gap_df['protein'] < required_nutrients['protein']
gap_df['fat_deficiency'] = gap_df['fat'] < required_nutrients['fat']
gap_df['carbohydrates_deficiency'] = gap_df['carbohydrates'] < required_nutrients['carbohydrates']


In [3]:
import pandas as pd

# Step 1: Load the final dataset
final_df = pd.read_csv("final_output.csv")  # Update path if needed

# Step 2: Define daily nutrient requirements (modify as needed)
required_nutrients = {
    'calories': 2000,
    'protein': 50,
    'fat': 70,
    'carbohydrates': 310
}

# Step 3: Perform Nutrient Gap Analysis
gap_df = final_df.copy()
gap_df['calories_gap'] = required_nutrients['calories'] - gap_df['calories']
gap_df['protein_gap'] = required_nutrients['protein'] - gap_df['protein']
gap_df['fat_gap'] = required_nutrients['fat'] - gap_df['fat']
gap_df['carbohydrates_gap'] = required_nutrients['carbohydrates'] - gap_df['carbohydrates']

# Step 4: Flag Deficiencies
gap_df['calories_deficiency'] = gap_df['calories'] < required_nutrients['calories']
gap_df['protein_deficiency'] = gap_df['protein'] < required_nutrients['protein']
gap_df['fat_deficiency'] = gap_df['fat'] < required_nutrients['fat']
gap_df['carbohydrates_deficiency'] = gap_df['carbohydrates'] < required_nutrients['carbohydrates']

# Step 5: Save as CSV
output_filename = "nutrient_gap_analysis.csv"
gap_df.to_csv(output_filename, index=False)

# Step 6: Generate Download Link (for Colab or Jupyter)
from IPython.display import FileLink, display
display(FileLink(output_filename))


/content/nutrient_gap_analysis.csv

Recommendation system

In [25]:
# STEP 1: INSTALL LIBRARIES
!pip install lightfm

# STEP 2: IMPORT LIBRARIES
import pandas as pd
import numpy as np
import random
from lightfm import LightFM
from lightfm.data import Dataset

# STEP 3: LOAD THE DATA
df = pd.read_csv("/content/nutrient_gap_analysis.csv")

# STEP 4: CALCULATE NUTRIENT SCORE (LOWER = BETTER)
df['nutrient_score'] = (
    df['calories_deficiency'].astype(int) +
    df['protein_deficiency'].astype(int) +
    df['fat_deficiency'].astype(int) +
    df['carbohydrates_deficiency'].astype(int)
)

# STEP 5: FILTER OUT USELESS RECIPES (WHERE NOTHING IS FIXED)
df_filtered = df[df['nutrient_score'] > 0].copy()

# STEP 6: CREATE RANK AND NORMALIZE TO RATING (HIGH = BETTER)
# STEP 6: CREATE RANK AND STRETCH TO RATING SCALE (BETTER VARIANCE)
df_filtered['nutrient_rank'] = df_filtered['nutrient_score'].rank(ascending=True)

# Stretch values to 0.3–1.0 to ensure rating variation
df_filtered['rating'] = 0.3 + (0.7 * (1 - df_filtered['nutrient_rank'] / df_filtered['nutrient_rank'].max()))

# Optional: Limit to top 1000 recipes only
df_filtered = df_filtered.sort_values(by='rating', ascending=False).head(1000)


# STEP 7: SIMULATE USERS
user_ids = ['user1', 'user2', 'user3', 'user4', 'user5']
df_filtered['user_id'] = [random.choice(user_ids) for _ in range(len(df_filtered))]

# STEP 8: BUILD INTERACTION MATRIX FOR LIGHTFM
dataset = Dataset()
dataset.fit(df_filtered['user_id'], df_filtered['recipe_name'])

(interactions, weights) = dataset.build_interactions([
    (row['user_id'], row['recipe_name'], row['rating']) for _, row in df_filtered.iterrows()
])

# STEP 9: TRAIN THE MODEL
model = LightFM(loss='warp')
model.fit(interactions, epochs=30, num_threads=2)

# STEP 10: MAPPING
user_id_map, _, item_id_map, _ = dataset.mapping()
item_labels = list(item_id_map.keys())

# STEP 11: FUNCTION TO RECOMMEND
def recommend_top_n(user, model, item_labels, n=5):
    user_index = user_id_map[user]
    scores = model.predict(user_index, np.arange(len(item_labels)))
    top_indices = np.argsort(-scores)[:n]
    return [(item_labels[i], round(scores[i], 2)) for i in top_indices]

# STEP 12: GET RECOMMENDATIONS FOR USER1
recommend_top_n('user1', model, item_labels)


[('pineapple upside down cake muffins', np.float32(0.23)),
 ('slow cooker brisket', np.float32(0.22)),
 ('mini taco bites', np.float32(0.16)),
 ('lasagna roll ups', np.float32(0.09)),
 ('chicken and sausage hunters style stew recipes', np.float32(0.08))]

evaluation matrices for recommendation system

🔍 Evaluation Metrics for the LightFM Model

📌 Precision@5: 1.0000
📌 Recall@5: 0.0252
📌 AUC Score: 1.0000
